In [5]:
import tensorflow as tf # models
import pandas as pd # data manipulation
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import datetime # manipulating dates
import os # accessing directory structure
import random # shuffling data
import cv2 # computer vision
import seaborn as sns # plotting
import sklearn # machine learning
from sklearn.model_selection import train_test_split # splitting data
from sklearn.metrics import confusion_matrix, roc_curve # model evaluation
from PIL import Image # manipulating images
import albumentations as A # data augmentation
import tensorflow_datasets as tfds # loading datasets
import tensorflow_probability as tfp # probabilistic models
from tensorflow.keras.models import Model # creating models
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D, Dense, Flatten, InputLayer, BatchNormalization, Input, Dropout, RandomFlip, RandomRotation, Resizing, Rescaling # creating layers
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, FalsePositives, FalseNegatives, TruePositives, TrueNegatives, Precision, Recall, AUC, binary_accuracy # creating evaluation metrics
from tensorflow.keras.optimizers import Adam # optimizers
from tensorflow.keras.callbacks import Callback, CSVLogger,EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau # callbacks
from tensorflow.keras.regularizers import L1, L2 # regularization
from tensorboard.plugins.hparams import api as hp # hyperparameter tuning
print(tf.__version__)

2.18.1


### Wandb Install, Login, Initialization and Configuration

In [6]:
%pip install wandb

   ---------------------------------------- 0.0/22.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/22.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/22.3 MB ? eta -:--:--
   - -------------------------------------- 0.8/22.3 MB 1.8 MB/s eta 0:00:13
   - -------------------------------------- 0.8/22.3 MB 1.8 MB/s eta 0:00:13
   -- ------------------------------------- 1.3/22.3 MB 1.7 MB/s eta 0:00:13
   --- ------------------------------------ 2.1/22.3 MB 2.0 MB/s eta 0:00:11
   ---- ----------------------------------- 2.6/22.3 MB 2.1 MB/s eta 0:00:10
   ------ --------------------------------- 3.4/22.3 MB 2.3 MB/s eta 0:00:09
   ------- -------------------------------- 3.9/22.3 MB 2.4 MB/s eta 0:00:08
   -------- ------------------------------- 4.7/22.3 MB 2.6 MB/s eta 0:00:07
   --------- ------------------------------ 5.2/22.3 MB 2.6 MB/s eta 0:00:07
   --------- ------------------------------ 5.5/22.3 MB 2.6 MB/s eta 0:00:07
   ----------- -----


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import wandb

In [ ]:
%wandb login

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: Enter your choice:wandb: WARNING Invalid choice
wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice:wandb: Enter your choice: